In [1]:
import intake
import satstac
import yaml

import intake_stac
intake_stac.__version__

'0.3.0.post58'

# Load an Intake Catalog

In [2]:
url = 'https://raw.githubusercontent.com/intake/intake-stac/main/intake_stac/tests/data/1.0.0/catalog/catalog.json'
stac_cat = intake.open_stac_catalog(url)
stac_cat

test:
  args:
    stac_obj: https://raw.githubusercontent.com/intake/intake-stac/main/intake_stac/tests/data/1.0.0/catalog/catalog.json
  description: ''
  driver: intake_stac.catalog.StacCatalog
  metadata:
    description: test catalog
    id: test
    stac_extensions: []
    stac_version: 1.0.0
    type: Catalog


# Serialize as a .YAML file

In [3]:
#stac_cat.walk(depth=2) # Works! type dict
#stac_cat.walk(depth=10).serialize() # Attribute Error, no attribute `serialize`

#print(stac_cat.yaml()) # Works! type str
#stac_cat.yaml(depth=1) # Type Error, unexpected keyword `depth`, this is our target api

#print(stac_cat.test.yaml()) # type st
#print(stac_cat.test) #type StacCatalog

In [4]:
#yaml.dump(stac_cat.walk(depth=2))
#print(yaml.dump(stac_cat)) # type str
print(yaml.dump(stac_cat.walk(depth=0))) # type str

#Missing source: and shouldn't have !!python/object:intake.catalog.local.LocalCatalogEntry to work

test: !!python/object:intake.catalog.local.LocalCatalogEntry
  args: []
  cls: intake.catalog.local.LocalCatalogEntry
  kwargs:
    name: test
    description: child catalog
    driver: !!python/name:intake_stac.catalog.StacCatalog ''
    catalog:
      cls: intake_stac.catalog.StacCatalog
      args:
      - https://raw.githubusercontent.com/intake/intake-stac/main/intake_stac/tests/data/1.0.0/catalog/catalog.json
      kwargs: {}
    args:
      stac_obj: https://raw.githubusercontent.com/intake/intake-stac/main/intake_stac/tests/data/1.0.0/catalog/child-catalog.json



In [5]:
with open('test2.yaml', 'w') as f:
    f.write(yaml.dump(stac_cat.walk(depth=9)))

Code from: https://github.com/intake/intake/blob/471488801bfa338dbfd91f9851b27580f4f3c3bb/intake/source/base.py#L242-L265showing how intake turns its catalogues into a dictionary with very specific keys before into a yaml.

```
def _yaml(self):
    import inspect
    kwargs = self._captured_init_kwargs.copy()
    meta = kwargs.pop('metadata', self.metadata) or {}
    kwargs.update(dict(zip(inspect.signature(self.__init__).parameters,
                       self._captured_init_args)))
    data = {
        'sources':
            {self.name: {
                'driver': self.classname,
                'description': self.description or "",
                'metadata': meta,
                'args': kwargs
            }}}
    return data

def yaml(self):
    """Return YAML representation of this data-source
    The output may be roughly appropriate for inclusion in a YAML
    catalog. This is a best-effort implementation
    """
    data = self._yaml()
    return yaml.dump(data, default_flow_style=False)
```

In [6]:
#`a` is a dictionary of `LocalCatalogEntry`s. I can pull out some attributes, but not others. Unsure why.

a = stac_cat.walk(depth=2)
#print(a)
#print(type(a))

print(a['test'])
#print(type(a['test']))

#print(a['test'].name)
#print(type(a['test'].name))

#print(a['test'].driver) #fails

name: test
container: catalog
plugin: ['stac_catalog']
driver: ['stac_catalog']
description: child catalog
direct_access: True
user_parameters: []
metadata: 
args: 
  stac_obj: https://raw.githubusercontent.com/intake/intake-stac/main/intake_stac/tests/data/1.0.0/catalog/child-catalog.json


In [7]:
#I'm trying to recreate the data dictionary from intake.yaml() without the references to self, but since driver can't be accessed it is stuck.

data = {
    'sources':
        {a['test'].name: {
            'driver': a['test'].driver,
            'description': a['test'].description or "",
            'metadata': a['test'].metadata,
            'args': a['test'].args
            }}}

AttributeError: 'LocalCatalogEntry' object has no attribute 'driver'

# Full Circle: Load Intake Catalogue from .YAML

In [8]:
catalog = intake.open_catalog('works.yaml') # Constructor Error

In [9]:
catalog2 = intake.open_catalog('test.yaml') # Constructor Error

ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/object:intake.catalog.local.LocalCatalogEntry'
  in "<unicode string>", line 1, column 7:
    test: !!python/object:intake.catalog.l ... 
          ^